<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#using logistic regression with mean squared error loss.
def sigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s
def forward(w, b, X, Y):
    
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    
    loss = (1/m) * np.sum(np.square(Y - A))
    
    
    dL = A - Y
    dA = A * (1 - A) * dL
    
    dw = (1/m)*np.dot(X,dA.T)
    db = 1/m*np.sum(dA)
    

 
    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [0]:
weights = forward(w0,b0,X,Y)